In [86]:
import sqlite3
from itertools import islice
from math import pi
import pandas as pd
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap


In [44]:
conn = sqlite3.connect("logs.db")
cursor = conn.cursor()


In [53]:
output_file("pie.html")

x = Report1()

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, title="Посетители из какой страны совершают больше всего действий на сайте", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)

In [ ]:
from bottle import route, run, template

@route('/hello/:name')
def index(name='World'):
    return template('<b>Hello {{name}}</b>!', name=name)

run(host='localhost', port=8080)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [13/Jan/2019 15:28:05] "GET /hello/world HTTP/1.1" 200 19


In [20]:
Посетители из какой страны совершают больше всего действий на сайте?
Посетители из какой страны чаще всего интересуются товарами из определенных категорий?
В какое время суток чаще всего просматривают определенную категорию товаров?
Какая нагрузка (число запросов) на сайт за астрономический час?
Товары из какой категории чаще всего покупают совместно с товаром из заданной категории?
Сколько брошенных (не оплаченных) корзин имеется за определенный период?
Какое количество пользователей совершали повторные покупки за определенный период?


SyntaxError: invalid syntax (<ipython-input-20-57ea45a0e479>, line 1)

In [90]:
# Посетители из какой страны совершают больше всего действий на сайте?
def Report1():
    cursor.execute("""select u.country, count(*) from users u, visits v where u.id = v.user_id group by country order by 2 desc""")
    result = {}
    for i in cursor.fetchall():
        result[i[0]] = i[1]
    cursor.execute("""select u.country, count(*) from users u, cart_history c where u.id = c.user_id group by u.country order by 2 desc""")
    for i in cursor.fetchall():
        result[i[0]] += i[1]
    cursor.execute("""select u.country, count(*) from users u, cart_history c, purchases p where u.id = c.user_id and p.cart_id = c.id group by u.country order by 2 desc""")
    for i in cursor.fetchall():
        result[i[0]] += i[1]
    cursor.execute("""select country, count(*) from users group by country order by 2 desc""")
    for i in cursor.fetchall():
        result[i[0]] /= i[1]
    del result[None]
    n_items = list(islice(result.items(), 12))
    result = {}
    for i in n_items:
        result[i[0]] = i[1]
    return result
    

In [2]:
cursor.execute("""select c.name, g.name from goods g,category_goods c where  c.id = g.category_id """)
print(cursor.fetchall())

[('canned_food', 'sprats'), ('canned_food', 'pate_of_tuna'), ('caviar', 'black_caviar'), ('frozen_fish', 'pike'), ('frozen_fish', 'shark'), ('fresh_fish', 'salmon'), ('canned_food', 'midii'), ('semi_manufactures', 'crab_meat'), ('frozen_fish', 'peljad'), ('fresh_fish', 'codfish'), ('fresh_fish', 'crucian'), ('semi_manufactures', 'stuffing_eel'), ('semi_manufactures', 'soup_set'), ('fresh_fish', 'tuna'), ('fresh_fish', 'marlene'), ('fresh_fish', 'herring'), ('semi_manufactures', 'squid_rings'), ('caviar', 'squash_caviar'), ('frozen_fish', 'salmon'), ('caviar', 'red_caviar'), ('fresh_fish', 'smelt'), ('semi_manufactures', 'salmon_cutlet'), ('frozen_fish', 'eel'), ('frozen_fish', 'carp')]


In [51]:
cursor.execute("""select country, count(*) from users group by country order by 2 desc""")
print(cursor.fetchall())

[('United States', 371), (None, 138), ('China', 91), ('Japan', 33), ('Germany', 31), ('United Kingdom', 30), ('France', 23), ('Brazil', 18), ('South Korea', 18), ('Italy', 13), ('Russia', 13), ('Australia', 12), ('Netherlands', 11), ('Canada', 10), ('Republic of Korea', 10), ('Sweden', 9), ('Belgium', 8), ('Egypt', 8), ('South Africa', 7), ('Norway', 6), ('Saudi Arabia', 6), ('Spain', 6), ('Mauritius', 5), ('Switzerland', 5), ('Taiwan', 5), ('Argentina', 4), ('Austria', 4), ('Chile', 4), ('India', 4), ('Indonesia', 4), ('Iran', 4), ('Malaysia', 4), ('Venezuela', 4), ('Finland', 3), ('Ireland', 3), ('Mexico', 3), ('Peru', 3), ('Poland', 3), ('Romania', 3), ('Thailand', 3), ('Algeria', 2), ('Bosnia and Herzegovina', 2), ('Colombia', 2), ('Denmark', 2), ('Ecuador', 2), ('Estonia', 2), ('Greece', 2), ('Israel', 2), ('Kazakhstan', 2), ('Luxembourg', 2), ('Nigeria', 2), ('Singapore', 2), ('Tunisia', 2), ('Turkey', 2), ('United Arab Emirates', 2), ('Vietnam', 2), ('Czechia', 1), ('Dominican R

In [89]:
def GetTimesOfDay(time):
    if time >= "00:00:00" and  time < "05:00:00":
        return 'Ночь'
    elif time >= "05:00:00" and  time < "12:00:00":
        return 'Утро'
    elif time >= "12:00:00" and  time < "18:00:00":
        return 'День'
    elif time >= "18:00:00" and  time <= "23:59:59":
        return 'Вечер'   

In [88]:
#В какое время суток чаще всего просматривают определенную категорию товаров?
#с 00:00:00  по 05:00:00 - ночь
#с 05:00:00  по 12:00:00 - утро
#с 12:00:00  по 18:00:00 - день
#с 18:00:00  по 00:00:00 - вечер
def Report2(category_id):
    cursor.execute("""select time(datetime) from visits where category_id = ?  """,[(category_id)])
    result = {}
    for i in cursor.fetchall():
        times = GetTimesOfDay(i[0])
        if times not in result:
            result[times] = 1
        else:
            result[times] += 1
    return result

In [87]:


output_file("bar_colormapped.html")

data = Report2(3)
time_of_days = ['Ночь', 'Утро', 'День', 'Вечер']
counts = [data['Ночь'],data['Утро'],data['День'],data['Вечер']]

source = ColumnDataSource(data=dict(time_of_days=time_of_days, counts=counts))

p = figure(x_range=fruits, plot_height=350, toolbar_location=None, title="В какое время суток чаще всего просматривают  категорию товаров")
p.vbar(x='time_of_days', top='counts', width=0.9, source=source, legend="time_of_days",
       line_color='white', fill_color=factor_cmap('time_of_days', palette=Spectral6, factors=fruits))

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 1500
p.legend.orientation = "vertical"
p.legend.location = "top_right"

show(p)

In [139]:
#Сколько брошенных (не оплаченных) корзин имеется за определенный период?
def Report3(date_start,date_finish):
    cursor.execute("""select c.id from cart_history c left join purchases p on c.id = p.cart_id  where p.cart_id is null and c.datetime between ? and ? """,[(date_start),(date_finish)])
    return len(cursor.fetchall())

In [140]:
date_start = '2018-08-01 00:28:36'
date_finish = '2018-08-14 09:53:56'
Report3(date_start,date_finish)

73